In [5]:
import requests
import matplotlib.pyplot as plt
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import us
import os
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [7]:
eia_key = os.environ['EIA_API_KEY']

# Access power plant data

In [4]:
params =  {
    "frequency": "monthly",
    "data": [
        "latitude",
        "longitude",
        "nameplate-capacity-mw",
        "operating-year-month"
    ],
    "facets": {
        "energy_source_code": [
            "BIT",
            "SUB"
        ]
    },
    "start": "2024-09",
    "end": None,
    "sort": [
        {
            "column": "period",
            "direction": "desc"
        }
    ],
    "offset": 0,
    "length": 5000
}

In [8]:
BASE_URL = "https://api.eia.gov/v2/"

dataset = "electricity/operating-generator-capacity"
route = dataset + "/data"

In [10]:
headers = {
            "X-Api-Key": eia_key,
            "X-Params": json.dumps(params),
        }

In [11]:
r = requests.get(BASE_URL+route, headers=headers)

In [14]:
response = r.json()['response']

print(f"Total records: {response['total']}")

data = response['data']

df = pd.DataFrame(data)
df.drop(columns=['stateName','sector','sectorName','entityid',
                    'balancing-authority-name','statusDescription', 
                    'entityName','nameplate-capacity-mw-units',
                    'energy_source_code','generatorid','status',
                    'unit', 'period'
                    ],
        inplace=True)

Total records: 428


In [18]:
df['nameplate-capacity-mw'] = df['nameplate-capacity-mw'].astype('float')

In [25]:
df = df.pivot_table(index=['stateid','plantid','balancing_authority_code','latitude','longitude'],
               columns=['technology'],
               values=['nameplate-capacity-mw']).reset_index(drop=False)